Jupyter notebook for the [February r/dataisbeautiful challenge](https://www.reddit.com/r/dataisbeautiful/comments/7vegvf/battle_dataviz_battle_for_the_month_of_february/)

Data set:[Legal status of Same-sex Marriage by US State and Year](https://github.com/zonination/samesmarriage/blob/master/ssm.csv)

I'm trying my hand at the [`bokeh`](https://bokeh.pydata.org/en/latest/) library. The goal here is to simply plot the progression from one legal state to the next for each state. The [heatmap](https://bokeh.pydata.org/en/latest/docs/user_guide/categorical.html#heat-maps) example was super helpful and instructive.

In [1]:
import pandas
from bokeh.io import show, output_notebook
from bokeh import palettes
from bokeh.models import ColumnDataSource, ColorBar
from bokeh.models.mappers import CategoricalColorMapper
from bokeh.plotting import figure
from bokeh.transform import transform

Here the data is loaded and cleaned up

- Indexed by state
- Abbreviations removed
- Collapsed into a 1D set to be used for plotting

In [2]:
data = pandas.read_csv('ssm.csv').set_index("State")
data.drop('abbrev', axis=1, inplace=True)
data.columns.name = "Year"
df = pandas.DataFrame(data.stack(), columns=['status']).reset_index()

Played around with some options on [colorbrewer](colorbrewer2.org) to find one that I like. I ended up with colorblind-safe Paired to represent the four legal standings.

In [3]:
LAW_TYPES = ['No Law', 'Statutory Ban', 'Constitutional Ban', 'Legal']
palette = palettes.Paired[4]
mapper = CategoricalColorMapper(palette=palette, factors=LAW_TYPES)

In [4]:
output_notebook()
TOOLS = "save"

Loading BokehJS ...

In [5]:
source = ColumnDataSource(df)
p = figure(plot_width=800, plot_height=1000, x_range=list(data.columns), 
           y_range=list(reversed(data.index)), 
           title='Same-sex Marriage Status',
           toolbar_location='right', tools=TOOLS, x_axis_location='above')
p.rect(y="State", x="Year", width=1, height=1, source=source,
       line_color=None, fill_color=transform('status', mapper),
       legend='status')
p.legend.background_fill_alpha = 0.6
show(p)